# Age, Gender and Expression Recognition – Training Notebook

This notebook trains and exports three convolutional neural network models for **age group**, **gender**, 
and **facial expression** recognition and prepares them for **on-device deployment** using TensorFlow Lite.

**Overview**
1. Install and import dependencies (TensorFlow, OpenCV, scikit-learn, etc.).  
2. Load and preprocess the **UTKFace** dataset for:
   - Binary gender classification (male / female)
   - Three-class age-group classification (child / adult / elderly)  
3. Build and train two MobileNetV2-based models for **gender** and **age group**, and compare CPU vs GPU training performance.  
4. Convert the trained age and gender models to **TensorFlow Lite** (`gender_model.tflite`, `age3_model.tflite`).  
5. Load and preprocess the **FER2013** dataset for 7-class facial expression recognition.  
6. Build and train a MobileNetV2-based model for **expression** (angry, disgust, fear, happy, neutral, sad, surprise).  
7. Convert the expression model to **TensorFlow Lite** (`expression7_model.tflite`) for integration into the iOS app.

You can run the notebook sequentially from top to bottom in Google Colab. Make sure that the UTKFace and FER2013
datasets are available in your Google Drive at the paths referenced in the code, or adjust the paths accordingly.


Install extra libs

In [2]:
!pip install "tensorflow==2.19.0" opencv-python scikit-learn matplotlib



In [3]:
import tensorflow as tf
print("TF version:", tf.__version__)


TF version: 2.19.0


Collab imports check tensor flow

In [ ]:
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

print("TensorFlow version:", tf.__version__)
print("GPU available?", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.19.0
GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


load toy data set

In [4]:
from tensorflow.keras.datasets import cifar10
import numpy as np

# Load CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
y_train = y_train.flatten()
y_test = y_test.flatten()

# Keep only classes 3 (cat) and 5 (dog)
train_mask = np.isin(y_train, [3, 5])
test_mask  = np.isin(y_test,  [3, 5])

X_train = X_train[train_mask]
y_train = y_train[train_mask]
X_test  = X_test[test_mask]
y_test  = y_test[test_mask]

# Map labels: 3 -> 0 (cat), 5 -> 1 (dog)
y_train = (y_train == 5).astype(np.int32)
y_test  = (y_test == 5).astype(np.int32)

# Normalize images to [0, 1]
X_train = X_train.astype("float32") / 255.0
X_test  = X_test.astype("float32") / 255.0

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)
print("Label counts (train):", np.bincount(y_train))


  4120576/170498071 ━━━━━━━━━━━━━━━━━━━━ 40s 0us/step

KeyboardInterrupt: 

Build the model

In [ ]:
from tensorflow.keras import layers, models

def build_model():
    model = models.Sequential([
        layers.Input(shape=(32, 32, 3)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # binary: cat vs dog
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

model = build_model()
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │       147,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 166,977 (652.25 KB)

 Trainable params: 166,977 (652.25 KB)

 Non-trainable params: 0 (0.00 B)

train and time the toy model (CPU run)

In [ ]:
import time

EPOCHS = 5
BATCH_SIZE = 64

start = time.time()
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)
elapsed = time.time() - start

print("\nTotal train time (s):", elapsed)
print("Time per epoch (s):", elapsed / EPOCHS)
print("Final val accuracy:", history.history['val_accuracy'][-1])


Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.5688 - loss: 0.6764 - val_accuracy: 0.6265 - val_loss: 0.6483
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.6518 - loss: 0.6250 - val_accuracy: 0.6620 - val_loss: 0.6074
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6908 - loss: 0.5816 - val_accuracy: 0.7155 - val_loss: 0.5651
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7205 - loss: 0.5464 - val_accuracy: 0.7060 - val_loss: 0.5555
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7359 - loss: 0.5290 - val_accuracy: 0.7195 - val_loss: 0.5418

Total train time (s): 14.647093772888184
Time per epoch (s): 2.929418754577637
Final val accuracy: 0.7195000052452087


Mount drive in colab

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


Unzip UTKFace into /content

In [ ]:
!ls "/content/drive/MyDrive/ML_datasets"


archive.zip


In [ ]:
!unzip "/content/drive/MyDrive/ML_datasets/archive.zip" -d "/content/UTKFace"


Archive:  /content/drive/MyDrive/ML_datasets/archive.zip
replace /content/UTKFace/UTKFace/100_0_0_20170112213500903.jpg.chip.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip "/content/drive/MyDrive/ML_datasets/archive.zip" \
       -d "/content/drive/MyDrive/ML_datasets/UTKFace_extracted"


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/utkface_aligned_cropped/crop_part1/34_1_0_20170109004755204.jpg.chip.jpg  
  inflating: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/utkface_aligned_cropped/crop_part1/34_1_0_20170111182452832.jpg.chip.jpg  
  inflating: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/utkface_aligned_cropped/crop_part1/34_1_1_20170103230340961.jpg.chip.jpg  
  inflating: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/utkface_aligned_cropped/crop_part1/34_1_1_20170104011329697.jpg.chip.jpg  
  inflating: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/utkface_aligned_cropped/crop_part1/34_1_1_20170104165020320.jpg.chip.jpg  
  inflating: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/utkface_aligned_cropped/crop_part1/34_1_1_20170108230211421.jpg.chip.jpg  
  inflating: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/utkface_aligned_cropp

In [ ]:
!ls "/content/drive/MyDrive/ML_datasets"
!ls "/content/drive/MyDrive/ML_datasets/UTKFace_extracted"

archive.zip  UTKFace_extracted
crop_part1  UTKFace  utkface_aligned_cropped


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
import os

search_root = "/content/drive/MyDrive/ML_datasets/UTKFace_extracted"
print("Searching for jpgs under:", search_root)

found = False
for root, dirs, files in os.walk(search_root):
    jpgs = [f for f in files if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    if jpgs:
        print("\n✅ Found image files here:")
        print("Folder:", root)
        print("Number of image files in this folder:", len(jpgs))
        print("Example file:", jpgs[0])
        found = True
        break

if not found:
    print("\n⚠️ No jpg/jpeg/png files found under", search_root)


Searching for jpgs under: /content/drive/MyDrive/ML_datasets/UTKFace_extracted

✅ Found image files here:
Folder: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/UTKFace
Number of image files in this folder: 23708
Example file: 90_0_0_20170111205428761.jpg.chip.jpg


Load UTKFace and build a gender dataset

In [11]:
import os, cv2
import numpy as np
from sklearn.model_selection import train_test_split

IMG_SIZE = 128  # resize faces to 128x128

def load_utkface_gender(data_dir, max_images=None):
    images = []
    labels = []  # 0 = male, 1 = female

    for i, fname in enumerate(os.listdir(data_dir)):
        if not fname.lower().endswith(".jpg"):
            continue

        if max_images is not None and i >= max_images:
            break

        try:
            # Filename format: age_gender_race_date.jpg
            parts = fname.split("_")
            if len(parts) < 4:
                continue
            age = int(parts[0])
            gender = int(parts[1])   # 0 or 1

            img_path = os.path.join(data_dir, fname)
            img = cv2.imread(img_path)
            if img is None:
                continue

            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            images.append(img)
            labels.append(gender)
        except Exception as e:
            # Skip files that don’t parse correctly
            continue

    X = np.array(images, dtype=np.float32) / 255.0
    y = np.array(labels, dtype=np.int64)

    print("Loaded images:", X.shape)
    print("Label distribution:", np.bincount(y))
    return train_test_split(X, y, test_size=0.2, random_state=42)

data_dir = "/content/drive/MyDrive/ML_datasets/UTKFace_extracted/UTKFace"
print("Using data_dir:", data_dir)

X_train, X_val, y_train, y_val = load_utkface_gender(data_dir, max_images=10000)
print("Train:", X_train.shape, "Val:", X_val.shape)


Using data_dir: /content/drive/MyDrive/ML_datasets/UTKFace_extracted/UTKFace
Loaded images: (9997, 128, 128, 3)
Label distribution: [6014 3983]
Train: (7997, 128, 128, 3) Val: (2000, 128, 128, 3)


Build MobileNetV2 gender model

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = 128  # make sure this matches your loader

def build_gender_model():
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet",
        pooling="avg"
    )
    base.trainable = False  # freeze backbone for now

    x = layers.Dropout(0.2)(base.output)
    output = layers.Dense(2, activation="softmax", name="gender")(x)

    model = models.Model(inputs=base.input, outputs=output)
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

gender_model = build_gender_model()
gender_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,260,546 (8.62 MB)

 Trainable params: 2,562 (10.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Train and time the gender model

In [13]:
import time

EPOCHS = 5
BATCH_SIZE = 32

start = time.time()
history = gender_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)
elapsed = time.time() - start

print("\nTotal train time (s):", elapsed)
print("Time per epoch (s):", elapsed / EPOCHS)
print("Final val accuracy:", history.history["val_accuracy"][-1])


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 48s 125ms/step - accuracy: 0.7099 - loss: 0.5972 - val_accuracy: 0.8170 - val_loss: 0.3968
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8157 - loss: 0.4123 - val_accuracy: 0.8165 - val_loss: 0.4167
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.8335 - loss: 0.3762 - val_accuracy: 0.8045 - val_loss: 0.4132
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - accuracy: 0.8313 - loss: 0.3832 - val_accuracy: 0.8440 - val_loss: 0.3628
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.8395 - loss: 0.3681 - val_accuracy: 0.8505 - val_loss: 0.3620

Total train time (s): 69.34472489356995
Time per epoch (s): 13.868944978713989
Final val accuracy: 0.8504999876022339


save keras model in collab

In [14]:
gender_model.save("gender_model.keras")
print("Saved gender_model.keras")

Saved gender_model.keras


Export to TensorFlow Lite

In [15]:
import tensorflow as tf

# 1. Create a *float* TFLite model (no quantization)
converter = tf.lite.TFLiteConverter.from_keras_model(gender_model)
# Do NOT set converter.optimizations here
tflite_model = converter.convert()

# 2. Save it
with open("gender_model.tflite", "wb") as f:
    f.write(tflite_model)

# 3. Download to your Mac
from google.colab import files
files.download("gender_model.tflite")



Saved artifact at '/tmp/tmp3_qelrkx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_157')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  135311567976016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311567977360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311549351120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311567978320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311567977168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311549350160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311549350544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311549350736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311549350928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135311549352080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1353115493

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download Model to Mac

In [1]:
from google.colab import files
files.download("gender_model.tflite")


FileNotFoundError: Cannot find file: gender_model.tflite

Load Age Classes (Child, Adult, Elderly)

In [1]:
import os, cv2
import numpy as np
from sklearn.model_selection import train_test_split

IMG_SIZE = 128

def load_utkface_age_3classes(root_dir, max_images=None):
    """
    0 = child   (age < 18)
    1 = adult   (18 <= age < 60)
    2 = elderly (age >= 60)
    """
    images = []
    labels = []
    count = 0

    for dirpath, dirnames, filenames in os.walk(root_dir):
        for fname in filenames:
            if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
                continue

            if max_images is not None and count >= max_images:
                break

            try:
                parts = fname.split("_")
                if len(parts) < 2:
                    continue

                age = int(parts[0])

                if age < 18:
                    label = 0  # child
                elif age < 60:
                    label = 1  # adult
                else:
                    label = 2  # elderly

                img_path = os.path.join(dirpath, fname)
                img = cv2.imread(img_path)
                if img is None:
                    continue

                img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                images.append(img)
                labels.append(label)
                count += 1
            except Exception:
                continue

        if max_images is not None and count >= max_images:
            break

    X = np.array(images, dtype=np.float32) / 255.0
    y = np.array(labels, dtype=np.int64)

    print("Loaded images:", X.shape)
    if len(y) > 0:
        counts = np.bincount(y, minlength=3)
        print("Label distribution [child, adult, elderly]:", counts)
    else:
        print("No labels loaded.")

    return train_test_split(X, y, test_size=0.2, random_state=42)


Load Data

In [4]:
data_dir = "/content/drive/MyDrive/ML_datasets/UTKFace_extracted/UTKFace"

X_train_age3, X_val_age3, y_train_age3, y_val_age3 = load_utkface_age_3classes(
    data_dir,
    max_images=10000  # adjust if needed
)

print("Train:", X_train_age3.shape, "Val:", X_val_age3.shape)


Loaded images: (10000, 128, 128, 3)
Label distribution [child, adult, elderly]: [1379 5963 2658]
Train: (8000, 128, 128, 3) Val: (2000, 128, 128, 3)


Build Age3 Model

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

IMG_SIZE = 128  # same as before

def build_age3_model():
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet",
        pooling="avg"
    )
    base.trainable = False  # freeze backbone for now

    x = layers.Dropout(0.2)(base.output)
    output = layers.Dense(3, activation="softmax", name="age3")(x)  # 0=child,1=adult,2=elderly

    model = models.Model(inputs=base.input, outputs=output)
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

age3_model = build_age3_model()
age3_model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,261,827 (8.63 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Train Age Model

In [6]:
import time

EPOCHS = 5      # you can increase later if training is fast
BATCH_SIZE = 32

start = time.time()
history_age3 = age3_model.fit(
    X_train_age3, y_train_age3,
    validation_data=(X_val_age3, y_val_age3),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)
elapsed = time.time() - start

print("\nTotal train time (s):", elapsed)
print("Time per epoch (s):", elapsed / EPOCHS)
print("Final val accuracy:", history_age3.history["val_accuracy"][-1])


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 121s 459ms/step - accuracy: 0.6460 - loss: 0.8340 - val_accuracy: 0.7755 - val_loss: 0.5220
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 129s 518ms/step - accuracy: 0.7593 - loss: 0.5673 - val_accuracy: 0.7870 - val_loss: 0.4967
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 130s 522ms/step - accuracy: 0.7774 - loss: 0.5238 - val_accuracy: 0.7910 - val_loss: 0.4907
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 130s 519ms/step - accuracy: 0.7747 - loss: 0.5236 - val_accuracy: 0.7760 - val_loss: 0.5201
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 147s 541ms/step - accuracy: 0.7785 - loss: 0.5023 - val_accuracy: 0.7920 - val_loss: 0.4872

Total train time (s): 663.0075747966766
Time per epoch (s): 132.60151495933533
Final val accuracy: 0.7919999957084656


Export Age3Model (age3_model.tflite)

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(age3_model)
# no optimizations → simple float model compatible with the iOS runtime
tflite_age3_model = converter.convert()

with open("age3_model.tflite", "wb") as f:
    f.write(tflite_age3_model)

print("Saved age3_model.tflite")

from google.colab import files
files.download("age3_model.tflite")


Saved artifact at '/tmp/tmp1k4xpa40'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  135636473732880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473734032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473734416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473733648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473732688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473734224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473734608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473735376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473734992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135636473731920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13563647373153

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Mount (FER2013)

In [11]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


unzip fer2013

In [7]:
FER_ZIP = "/content/drive/MyDrive/ML_datasets/FER2013/fer2013.zip"
FER_DIR = "/content/FER2013"

# Create target dir (if not exists)
!mkdir -p "$FER_DIR"

# Unzip (quiet mode)
!unzip -q "$FER_ZIP" -d "$FER_DIR"

# See what's inside
!ls "$FER_DIR"


replace /content/FER2013/test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: test  train


check folder

In [4]:
FER_DIR = "/content/FER2013"
!ls "$FER_DIR/train"


angry  disgust	fear  happy  neutral  sad  surprise


load data

In [2]:
import os, cv2
import numpy as np

IMG_SIZE = 128  # same as your other models

# FER2013 7 classes (expected folder names under train/ and test/)
CLASS_NAMES_EXPR = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
CLASS_TO_IDX_EXPR = {name: i for i, name in enumerate(CLASS_NAMES_EXPR)}

FER_TRAIN_DIR = "/content/FER2013/train"
FER_TEST_DIR  = "/content/FER2013/test"

def load_fer_split(split_dir, max_images_per_class=None):
    images = []
    labels = []

    for class_name in CLASS_NAMES_EXPR:
        class_dir = os.path.join(split_dir, class_name)
        if not os.path.isdir(class_dir):
            print(f"⚠️ Missing folder {class_dir}, skipping this class.")
            continue

        files = [f for f in os.listdir(class_dir)
                 if f.lower().endswith((".jpg", ".jpeg", ".png"))]

        # Optional cap per class (to keep things fast)
        if max_images_per_class is not None:
            files = files[:max_images_per_class]

        for fname in files:
            img_path = os.path.join(class_dir, fname)
            # FER2013 is grayscale; read as grayscale
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            # Resize to 128x128
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            # Convert to 3-channel RGB for MobileNetV2
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            images.append(img)
            labels.append(CLASS_TO_IDX_EXPR[class_name])

    X = np.array(images, dtype=np.float32) / 255.0
    y = np.array(labels, dtype=np.int64)

    print(f"Loaded from {split_dir}: {X.shape}")
    if len(y) > 0:
        counts = np.bincount(y, minlength=len(CLASS_NAMES_EXPR))
        print("Label distribution [angry, disgust, fear, happy, neutral, sad, surprise]:", counts)
    else:
        print("No labels loaded.")

    return X, y


Build 7 class expression Model

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_expression7_model():
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights="imagenet",
        pooling="avg"
    )
    base.trainable = False  # freeze backbone for now

    x = layers.Dropout(0.3)(base.output)
    output = layers.Dense(7, activation="softmax", name="expression7")(x)

    model = models.Model(inputs=base.input, outputs=output)
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

expr7_model = build_expression7_model()
expr7_model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,266,951 (8.65 MB)

 Trainable params: 8,967 (35.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Free RAM

In [5]:
# Only run this if you don't need to re-train UTKFace models right now
vars_to_delete = [
    "X_train", "X_val", "y_train", "y_val",          # gender
    "X_train_age3", "X_val_age3", "y_train_age3", "y_val_age3"  # age 3-class
]

for v in vars_to_delete:
    if v in globals():
        del globals()[v]

import gc
gc.collect()


308

In [8]:
import os, cv2
import numpy as np

IMG_SIZE = 128  # same as your other models

# FER2013 7 classes (folder names under train/ and test/)
CLASS_NAMES_EXPR = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
CLASS_TO_IDX_EXPR = {name: i for i, name in enumerate(CLASS_NAMES_EXPR)}

FER_DIR = "/content/FER2013"
FER_TRAIN_DIR = os.path.join(FER_DIR, "train")
FER_TEST_DIR  = os.path.join(FER_DIR, "test")

def load_fer_split(split_dir, max_images_per_class=None):
    images = []
    labels = []

    for class_name in CLASS_NAMES_EXPR:
        class_dir = os.path.join(split_dir, class_name)
        if not os.path.isdir(class_dir):
            print(f"⚠️ Missing folder {class_dir}, skipping this class.")
            continue

        files = [f for f in os.listdir(class_dir)
                 if f.lower().endswith((".jpg", ".jpeg", ".png"))]

        if max_images_per_class is not None:
            files = files[:max_images_per_class]

        for fname in files:
            img_path = os.path.join(class_dir, fname)
            # FER2013 is grayscale; read as grayscale
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            # Resize to 128x128
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            # Convert to 3-channel RGB for MobileNetV2
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

            images.append(img)
            labels.append(CLASS_TO_IDX_EXPR[class_name])

    X = np.array(images, dtype=np.float32) / 255.0
    y = np.array(labels, dtype=np.int64)

    print(f"Loaded from {split_dir}: {X.shape}")
    if len(y) > 0:
        counts = np.bincount(y, minlength=len(CLASS_NAMES_EXPR))
        print("Label distribution [angry, disgust, fear, happy, neutral, sad, surprise]:", counts)
    else:
        print("No labels loaded.")

    return X, y


load data

In [9]:
# Limit per class to keep RAM down
MAX_PER_CLASS = 1500   # if too big, try 1000 or 500

X_train_expr, y_train_expr = load_fer_split(FER_TRAIN_DIR, max_images_per_class=MAX_PER_CLASS)
X_val_expr,   y_val_expr   = load_fer_split(FER_TEST_DIR,  max_images_per_class=MAX_PER_CLASS)

print("Train:", X_train_expr.shape, "Val:", X_val_expr.shape)


Loaded from /content/FER2013/train: (9436, 128, 128, 3)
Label distribution [angry, disgust, fear, happy, neutral, sad, surprise]: [1500  436 1500 1500 1500 1500 1500]
Loaded from /content/FER2013/test: (6904, 128, 128, 3)
Label distribution [angry, disgust, fear, happy, neutral, sad, surprise]: [ 958  111 1024 1500 1233 1247  831]
Train: (9436, 128, 128, 3) Val: (6904, 128, 128, 3)


Train 7 Expression Model

In [13]:
import time

EPOCHS = 5      # can increase later
BATCH_SIZE = 32

start = time.time()
history_expr7 = expr7_model.fit(
    X_train_expr, y_train_expr,
    validation_data=(X_val_expr, y_val_expr),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)
elapsed = time.time() - start

print("\nTotal train time (s):", elapsed)
print("Time per epoch (s):", elapsed / EPOCHS)
print("Final val accuracy:", history_expr7.history["val_accuracy"][-1])


Epoch 1/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 53s 121ms/step - accuracy: 0.2477 - loss: 2.1077 - val_accuracy: 0.3889 - val_loss: 1.5978
Epoch 2/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 0.3662 - loss: 1.6656 - val_accuracy: 0.4067 - val_loss: 1.5262
Epoch 3/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.4119 - loss: 1.5508 - val_accuracy: 0.4408 - val_loss: 1.4560
Epoch 4/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 9s 31ms/step - accuracy: 0.4265 - loss: 1.5043 - val_accuracy: 0.4434 - val_loss: 1.4721
Epoch 5/5
295/295 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.4363 - loss: 1.4631 - val_accuracy: 0.4303 - val_loss: 1.4863

Total train time (s): 87.68488764762878
Time per epoch (s): 17.536977529525757
Final val accuracy: 0.4303302466869354


Export Model

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model(expr7_model)
tflite_expr7 = converter.convert()

with open("expression7_model.tflite", "wb") as f:
    f.write(tflite_expr7)

print("Saved expression7_model.tflite")

from google.colab import files
files.download("expression7_model.tflite")


Saved artifact at '/tmp/tmpqp_pu09j'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  135246176397904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176399056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176399440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176398672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176397712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176399248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176399632: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176400400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176400016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135246176396944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13524617639656

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>